In [ ]:
var = 'total score'
a = 'predictions ' + var + '.csv'
print(a)
print('predictions total score.csv')

predictions total score.csv
predictions total score.csv


In [ ]:
var = "total score"
a = "predictions " + var + ".csv"
print(a)
print("predictions total score.csv")

predictions total score.csv
predictions total score.csv


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 17 20:06:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install tensorflow_text

In [ ]:
!pip install tf-models-official

In [ ]:
# -*-coding:utf8 -*-
import tensorflow as tf
# print("TensorFlow version:", tf.__version__)

import os

import pandas as pd
import numpy as np

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.python import keras
from keras.layers import Dense, Flatten, Conv2D
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.applications.vgg16 import VGG16
from keras import Model, Input, layers


import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

from keras import Model, Input, layers, regularizers
from keras.models import load_model
from keras import activations

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Vehicle Rating Prediction/new_images_with_folder
!unzip "new_images_with_folder.zip"

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Vehicle Rating Prediction/new_interior_images_with_folder
!unzip "new_interior_images_with_folder.zip"

In [ ]:

# change to your personal project address

%cd /content/drive/MyDrive/Colab Notebooks/Vehicle Rating Prediction

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/across-attention/total score


## **1 Data Processing**

In [ ]:
var = "total score"
# var = "safety score"
# var = "performance score"
# var = "interior score"
# var = "critics score"

In [ ]:
# 1 first we need to get the total score information from the csv file
# read info_data
file_name = "parametric data 2571 normalize " + var + ".csv"
info_data = pd.read_csv(file_name)
# get numpy matrix which only contains data (do not contain the title)
info_data = np.array(info_data)
print(info_data.shape)  # (2571, 303)
print(len(info_data))
print(info_data.shape[1])

(2571, 310)
2571
310


In [ ]:
# train data shuffle index
num1 = 2055
idx1 = tf.range(num1)
idx1 = tf.random.shuffle(idx1)
# print(idx1)
# print(idx1[0])
with tf.compat.v1.Session():
    index1 = idx1.numpy()
# print(index1.shape)
# print(index1[0])

# validation data shuffle index
num2 = 258
idx2 = tf.range(num2)
idx2 = tf.random.shuffle(idx2)
# print(idx2)
# print(idx2[0])
with tf.compat.v1.Session():
    index2 = idx2.numpy()
# print(index2.shape)
# print(index2[0])

# test data shuffle index
num3 = 258
idx3 = tf.range(num3)
idx3 = tf.random.shuffle(idx3)
# print(idx3)
# print(idx3[0])
with tf.compat.v1.Session():
    index3 = idx3.numpy()
# print(index3.shape)
# print(index3[0])

In [ ]:
# assign the parametric data
x_train_tab = np.zeros((num1, 302))
y_train_tab = np.zeros((num1, 1))
for i in range(num1):
    x_train_tab[i, :] = np.array(info_data[index1[i], 2:304], dtype=float)
    y_train_tab[i] = np.array(info_data[index1[i], 304], dtype=float)
x_train_tab = tf.convert_to_tensor(x_train_tab)
y_train_tab = tf.convert_to_tensor(y_train_tab)
y_train = y_train_tab
print(y_train.shape)

x_validation_tab = np.zeros((num2, 302))
y_validation_tab = np.zeros((num2, 1))
for i in range(num2):
    x_validation_tab[i, :] = np.array(info_data[index2[i] + num1, 2:304], dtype=float)
    y_validation_tab[i] = np.array(info_data[index2[i] + num1, 304], dtype=float)
x_validation_tab = tf.convert_to_tensor(x_validation_tab)
y_validation_tab = tf.convert_to_tensor(y_validation_tab)
y_validation = y_validation_tab
print(y_validation.shape)

x_test_tab = np.zeros((num3, 302))
y_test_tab = np.zeros((num3, 1))
for i in range(num3):
    x_test_tab[i, :] = np.array(info_data[index3[i] + num1 + num2, 2:304], dtype=float)
    y_test_tab[i] = np.array(info_data[index3[i] + num1 + num2, 304], dtype=float)
x_test_tab = tf.convert_to_tensor(x_test_tab)
y_test_tab = tf.convert_to_tensor(y_test_tab)
y_test = y_test_tab
print(y_test.shape)

(2055, 1)
(258, 1)
(258, 1)


In [ ]:
# assign the text data
var_name = 'data split ' + var

sketch1 = pd.read_csv('text_data.csv', encoding='latin1')
# print(sketch1.shape)
# print(sketch1)
# print(sketch1[var])
train_df = sketch1[sketch1[var_name] == 1]
val_df = sketch1[sketch1[var_name] == 2]
test_df = sketch1[sketch1[var_name] == 3]
# print(train_df.shape)
# print(train_df)

sketch2 = train_df.astype({"text": str})
text1 = list(sketch2['text'])

sketch3 = val_df.astype({"text": str})
text2 = list(sketch3['text'])

sketch4 = test_df.astype({"text": str})
text3 = list(sketch4['text'])


train_text = [text1[i] for i in index1]
x_train_text = tf.constant(train_text)

validation_text = [text2[i] for i in index2]
x_validation_text = tf.constant(validation_text)

test_text = [text3[i] for i in index3]
x_test_text = tf.constant(test_text)
# print(len(train_text))
# print(train_text[0])

In [ ]:
# assign the exterior image data
image = np.zeros((290, 448, 3))
x_train = np.zeros((num1, 290, 448, 3))
x_validation = np.zeros((num2, 290, 448, 3))
x_test = np.zeros((num3, 290, 448, 3))
# image = image_data[0]

# train data
for i in range(num1):
    folder_path = r'new_images_with_folder/' + info_data[index1[i]][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_train[i, :, :, :] = img_plt / 255.0

# validation data
for i in range(num2):
    folder_path = r'new_images_with_folder/' + info_data[index2[i]+num1][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_validation[i, :, :, :] = img_plt / 255.0

# test data
for i in range(num3):
    folder_path = r'new_images_with_folder/' + info_data[index3[i]+num1+num2][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_test[i, :, :, :] = img_plt / 255.0

x_train_ex_img = tf.convert_to_tensor(x_train)
x_validation_ex_img = tf.convert_to_tensor(x_validation)
x_test_ex_img = tf.convert_to_tensor(x_test)

In [ ]:
# assign the interior image data
image = np.zeros((300, 448, 3))
x_train = np.zeros((num1, 300, 448, 3))
x_validation = np.zeros((num2, 300, 448, 3))
x_test = np.zeros((num3, 300, 448, 3))
# image = image_data[0]

# train data
for i in range(num1):
    folder_path = r'new_interior_images_with_folder/' + info_data[index1[i]][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_train[i, :, :, :] = img_plt / 255.0


# validation data
for i in range(num2):
    folder_path = r'new_interior_images_with_folder/' + info_data[index2[i]+num1][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_validation[i, :, :, :] = img_plt / 255.0


# test data
for i in range(num3):
    folder_path = r'new_interior_images_with_folder/' + info_data[index3[i]+num1+num2][1]
    dirs = os.listdir(folder_path)
    print(i)
    if len(dirs) > 0:
        # only one total picture
        dirpath = folder_path + '/' + dirs[0]  # get the angular front view of the car
        img = Image.open(dirpath)
        img_plt = np.array(img)
        x_test[i, :, :, :] = img_plt / 255.0


x_train_in_img = tf.convert_to_tensor(x_train)
x_validation_in_img = tf.convert_to_tensor(x_validation)
x_test_in_img = tf.convert_to_tensor(x_test)

## **Load unimodal model**

In [ ]:
# construct MML model
adam_optimizer = Adam(learning_rate=0.001)
rms_prop_optimizer = RMSprop(learning_rate=0.001)
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=False)

In [ ]:
# load the pretrain model


# MLP
MLPmodel = tf.keras.models.load_model('model weight/' + var + '_Par.h5')
# for layer in MLPmodel.layers:
#   layer._name = layer._name + "_b"
MLPmodel.summary()

# Bert
model_name = 'model weight/' + var + '_Text.h5'
Bertmodel = tf.keras.models.load_model(model_name, custom_objects={'KerasLayer': hub.KerasLayer})
# for layer in Bertmodel.layers:
#   layer._name = layer._name + "_c"
Bertmodel.summary()

# Ex_CNN
Ex_CNNmodel = tf.keras.models.load_model('model weight/' + var + '_Ex_Img.h5')
# for layer in Ex_CNNmodel.layers:
#   layer._name = layer._name + "_a"
Ex_CNNmodel.summary()

# In_CNN
In_CNNmodel = tf.keras.models.load_model('model weight/' + var + '_In_Img.h5')
# for layer in In_CNNmodel.layers:
#   layer._name = layer._name + "_d"
In_CNNmodel.summary()

Model: "Tabular_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tab_dense1 (Dense)          (None, 302)               91506     
                                                                 
 tab_dropout1 (Dropout)      (None, 302)               0         
                                                                 
 tab_dense2 (Dense)          (None, 100)               30300     
                                                                 
 tab_dense3 (Dense)          (None, 1)                 101       
                                                                 
Total params: 121,907
Trainable params: 121,907
Non-trainable params: 0
_________________________________________________________________


Model: "Text_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                 

## **get prediction result**

In [ ]:
result1 = MLPmodel.predict(x_train_tab)

result2 = Bertmodel.predict(x_train_text)

result3 = Ex_CNNmodel.predict(x_train_ex_img)

result4 = In_CNNmodel.predict(x_train_in_img)


65/65 [==============================] - 5s 70ms/step


In [ ]:
result5 = MLPmodel.predict(x_test_tab)

result6 = Bertmodel.predict(x_test_text)

result7 = Ex_CNNmodel.predict(x_test_ex_img)

result8 = In_CNNmodel.predict(x_train_in_img)

In [ ]:
matrix = np.zeros((2313, 5))
for i in range(2055):
    matrix[i, 0] = result1[i]
    matrix[i, 1] = result2[i]
    matrix[i, 2] = result3[i]
    matrix[i, 3] = result4[i]
    matrix[i, 4] = y_train[i]
for i in range(258):
    matrix[i+2055, 0] = result5[i]
    matrix[i+2055, 1] = result6[i]
    matrix[i+2055, 2] = result7[i]
    matrix[i+2055, 3] = result8[i]
    matrix[i+2055, 4] = y_test[i]

In [ ]:
import csv

In [ ]:
headers = ['MLP', 'Bert', 'Ex_CNN', 'In_CNN', 'True Value']
file_name1 = 'predictions_' + var + '.csv'
with open(file_name1, 'w', newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(matrix)

f.close()